<h1 align='center'> Buy low, sell high - a walkthrough </h1>

Why hello there! Ready to make a shitload of money on the stock market? Good. Here is how you do it.

## The Classes

This framework is made up of two classes of Python objects: Portfolios and Stocks. 

- A <b>Portfolio</b> object is defined by its capital and Stocks. 
  - Capital can be used to purchase shares of a Stock object and Stock shares can be sold for capital. 
  - Portfolio objects contain a ledger that records details from each transactions.
- A <b>Stock</b> object is defined by its ticker symbol. 
  - When a stock is added to a portfolio, an instance of a AlphaVantage TimeSeries object is created and both daily and intradaily data is gathered.
  - The gathered data contains the open, close, high, and low values per share and the total trade volume and is used to define the buy and sell prices for a Stock at a particular time.

## Examining the data

Before we dive into buying and selling stock, let's take a minute to understand the data the we are using. Each Stock object comes with a large amount of data that can be manipulated and aggregated at different intervals, so let's explore a little bit!

In order to start creating objects and calling functions, we need to import the classes from the python file.

In [1]:
from classes import *

Let's create a Stock object and gather it's data. Normally, this will happen behind the scenes when a Stock is added to a Portfolio, but let's go through the details here.

In [2]:
Google = Stock('GOOGL')
Google.gatherDaily()
Google.gatherIntra()

Both of the gather functions have parameters that can be adjusted. Calling the function with no parameters will use the defaults.

- <b>gatherDaily(outputsize)</b>: Returns a Pandas DataFrame with open, close, high, low, and trade volume across days
   - <b>outputsize</b> defines how much data is returned: "compact" (default) returns the last 100 datapoints, "full" returns all available data
- <b>gatherIntra(interval, outputsize)</b>: Returns a Pandas DataFrame with open, close, high, low data, and trade volume within a day at the given interval
   - <b>interval</b> defines time difference between datapoints: '1min', '5min', '15min', '30min' (default), '60min'
   - <b>outputsize</b> defines how much data is returned: "compact" (default) returns the last 100 datapoints, "full" returns all available data

These DataFrames are stored as variables on the Stock object:

In [3]:
Google.daily.head()

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2020-03-20,1130.90,1138.48,1062.10,1068.21,4143897.0
2020-03-23,1056.37,1066.91,1008.87,1054.13,4183629.0
2020-03-24,1103.98,1133.33,1086.43,1130.01,3623435.0
2020-03-25,1124.58,1146.17,1083.01,1101.62,4516162.0
2020-03-26,1114.72,1171.48,1092.03,1162.92,3834422.0


In [4]:
Google.intra.head()

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2020-08-04 04:30:00,1483.0,1483.0,1483.0,1483.00,171.0
2020-08-04 07:30:00,1480.0,1480.0,1480.0,1480.00,687.0
2020-08-04 08:30:00,1483.0,1483.0,1478.0,1478.00,779.0
2020-08-04 09:00:00,1476.0,1476.0,1476.0,1476.00,145.0
2020-08-04 09:30:00,1480.0,1486.1,1479.0,1484.17,7749.0


These DataFrames are used to determine the buy/sell prices at a particular time, among other things. Each Stock has functions <b>buyPrice</b> and <b>sellPrice</b> that will return the price for a given index. You can pass in an integer index (e.g., 0 for the first line in the daily dataset) or an actual date/time (e.g. '2020-07-28 15:30:00 for the first line in the intra dataset). 

The value returned is <u>the open price for the next index in the dataframe</u>.

In [5]:
sellPrice = Google.sellPrice(0, isDate=0)

print("The first sell price for Google is", sellPrice)

The first sell price for Google is 1480.0


In [6]:
datetime = Google.intra.index[0]
buyPrice = Google.buyPrice(datetime, isDate=1)

print('The first date in the dateset is', datetime)
print('The buy price for Google on {} is {}'.format(datetime,buyPrice))

The first date in the dateset is 2020-08-04 04:30:00
The buy price for Google on 2020-08-04 04:30:00 is 1480.0


There is a similar function for trade volume:

In [7]:
volume = Google.volume(3, isDate=0)

print('The total volume for the third row in the dataset is', volume)

The total volume for the third row in the dataset is 145.0


## Creating a portfolio and buying stock

Okay, now that we have an understanding of what we're working with, let's get started. 

The first thing we will need to do is create an instance of a Portfolio. I've been watching a bunch of Archer lately, so let's set Sterling up with a new portfolio using the $100k his mommy gave him.

In [8]:
Archer = Portfolio(100000)

Now that we have a portfolio, we can buy some stock! 

Sterling would love to pour his money into his favorite things: booze and hookers. Thankfully, we were able to satisfy his urges and convince him otherwise by using Woodhouse's smack needles to inject 8 gils of liquor right into his veins and finding the cheapiest, dirtiest whore we could find off of the street. I'm talking Frank Reynolds level dirty whore. But I digress...

Archer decided to put his money into gold, water, and keep a bit left as available captial. 

Now, we can choose to buy stocks by either specifying the number of shares or the dollar amount. This is driven by the variable <b>isShares</b>, which defaults to number of shares. For now, we will split Archer's portfolio out by dollar amount, with 40k going to GOLD, 40k going to AWK, and 20k remaining as capital.

In [9]:
Archer.buy('GOLD', 40000, isShares=0)

1340.48 shares of GOLD were purchased for $40000


In [10]:
Archer.buy('AWK', 40000, isShares=0)

272.17 shares of AWK were purchased for $40000


We can see Archer's current stock holdings and capital with the <b>shares</b> and <b>capital</b> variables:

In [11]:
Archer.shares

{'GOLD': 1340.48, 'AWK': 272.17}

In [12]:
Archer.capital

20000

Additionally, you can see a history of Archer's transactions with the function <b>getLedger</b>, which returns a list of transactions in the form of a Pandas DataFrame:

In [13]:
Archer.getLedger()

,date,action,symbol,Δshares,Δdollars,capital
0,2020-08-05 18:30:00,buy,GOLD,1340.48,-40000,60000
1,2020-07-31 12:00:00,buy,AWK,272.17,-40000,20000


On thing you might notice is that the dates are different in the ledger above, even though we bought the stock at the same time. This is because when we called the <b>buy</b> function, <u>we did not specify a date</u>. If no date is specified, the stock is purchased at the open price of the first point in the dataset. Our two stocks have a different first entry in their datasets, so the dates are different in the ledger.

## Selling shares

Now, Mr. Archer is getting a little impatient and would like his shares liquidated so that we can go buy a bunch of sand to rub in Woodhouse's eyes. What kind of sand, you ask? Coarse.]]

We can get rid of Sterling's stock with the <b>sell</b> function. In the sell function, you can specify the stock you want to sell, the amount in shares or dollars, and the index/date/time in which to sell. If you don't put an amount all holdings are sold and if you don't put in an index the last one is used.

In [14]:
Archer.sell('GOLD')

No value entered, all shares will be sold
1340.48 shares of GOLD were sold for $38632.63


In [15]:
Archer.sell('AWK')

No value entered, all shares will be sold
272.17 shares of AWK were sold for $40583.27


In [16]:
Archer.capital

99215.9

In [17]:
Archer.getLedger()

,date,action,symbol,Δshares,Δdollars,capital
0,2020-08-05 18:30:00,buy,GOLD,1340.48,-40000.00,60000.00
1,2020-07-31 12:00:00,buy,AWK,272.17,-40000.00,20000.00
2,2020-08-10 19:30:00,sell,GOLD,-1340.48,38632.63,58632.63
3,2020-08-10 16:00:00,sell,AWK,-272.17,40583.27,99215.90


In [20]:
if Archer.capital > 100000:
    print('Wow, the world\'s greatest spy made a profit! What a surprise...')
else:
    print('Archer lost money. What an asshole.')

Archer lost money. What an asshole.


## Come back next time for instructions on how to use this framework to implement an algorithm!